# Saved Snippets
This script holds random snippets of code that I may need again, but are cluttering the main scripts.

## Plot Yeo Functional Connectivity Using Seaborn

Based off of:
- https://seaborn.pydata.org/examples/heat_scatter.html
- https://seaborn.pydata.org/examples/many_pairwise_correlations.html

In [ ]:
# Search for all functional connectivity files and read them
fc_path = '/imaging3/owenlab/bpho/python_yeo_fc'
fc_paths = glob.glob(fc_path + '/**/yeo_fc.npy', recursive=True)

fcs = {}
for path in fc_paths:
    subject_id = get_subject_from_path(path)
    fcs[subject_id] = np.load(path)
print("Number of functional connectivity:", len(fcs))
print("Number of features (connections):", fcs["NDARAP912JK3"].shape)

In [ ]:
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(fcs["NDARAP912JK3"], cmap=cmap, center=0,
            square=True)

In [ ]:
sns.set_theme(style="whitegrid")

# Load the brain networks dataset, select subset, and collapse the multi-index
df = sns.load_dataset("brain_networks", header=[0, 1, 2], index_col=0)

used_networks = [1, 5, 6, 7, 8, 12, 13, 17]
used_columns = (df.columns
                  .get_level_values("network")
                  .astype(int)
                  .isin(used_networks))
df = df.loc[:, used_columns]

df.columns = df.columns.map("-".join)

# Compute a correlation matrix and convert to long-form
corr_mat = df.corr().stack().reset_index(name="correlation")

# Draw each cell as a scatter point with varying size and color
g = sns.relplot(
    data=fcs["NDARAP912JK3"],
    x="level_0", y="level_1", hue="correlation", size="correlation",
    palette="vlag", hue_norm=(-1, 1), edgecolor=".7",
    height=10, sizes=(50, 250), size_norm=(-.2, .8),
)

# Tweak the figure to finalize
g.set(xlabel="", ylabel="", aspect="equal")
g.despine(left=True, bottom=True)
g.ax.margins(.02)
for label in g.ax.get_xticklabels():
    label.set_rotation(90)
for artist in g.legend.legendHandles:
    artist.set_edgecolor(".7")

## Custom PLSRW

In [ ]:
from scipy.linalg import pinv2
from sklearn.metrics import r2_score

In [ ]:
def _center_scale_xy(X, Y, scale=True):
    """ Center X, Y and scale if the scale parameter==True

    Returns
    -------
        X, Y, x_mean, y_mean, x_std, y_std
    """
    # center
    x_mean = X.mean(axis=0)
    X -= x_mean
    y_mean = Y.mean(axis=0)
    Y -= y_mean
    # scale
    if scale:
        x_std = X.std(axis=0, ddof=1)
        x_std[x_std == 0.0] = 1.0
        X /= x_std
        y_std = Y.std(axis=0, ddof=1)
        y_std[y_std == 0.0] = 1.0
        Y /= y_std
    else:
        x_std = np.ones(X.shape[1])
        y_std = np.ones(Y.shape[1])
    return X, Y, x_mean, y_mean, x_std, y_std


class PLSRW():
    
    def __init__(self, n_components=2, scale=True, reg=0.01):
        self.n_components=n_components
        self.scale = scale
        self.reg = reg
    
    def _calc_dist(self, X, Y):
        dist = []
        
        for feature in range(X.shape[1]):
            feature_dist = np.linalg.norm(Y - X[:, feature])
            dist.append(feature_dist)
        
        return np.array(dist)
    
    def fit(self, X, Y):
        Y = Y.astype('float64')
        if Y.ndim == 1:
            Y = Y.reshape(-1, 1)
        
        n = X.shape[0]
        p = X.shape[1]
        q = Y.shape[1]
        
        n_components = self.n_components
        reg = self.reg
        eps = np.finfo(X.dtype).eps
        Y_eps = np.finfo(Y.dtype).eps
        
        self.x_weights_ = np.zeros((p, n_components))  # U
        self._x_scores = np.zeros((n, n_components))  # Xi
        self.x_loadings_ = np.zeros((p, n_components))  # Gamma
        self.y_loadings_ = np.zeros((q, n_components))  # Delta
        
        # Scale (in place)
        Xk, Yk, self._x_mean, self._y_mean, self._x_std, self._y_std = (
            _center_scale_xy(X, Y, self.scale))
        Yk_mask = np.all(np.abs(Yk) < 10 * Y_eps, axis=0)
        Yk[:, Yk_mask] = 0.0
        
        for k in range(n_components):
            # Compute the regularization matrix
            d = self._calc_dist(Xk, Yk)
            D = np.diag(d)
            print(reg * (D.T @ D))
            print((Xk.T @ Xk))
            
            # Compute the PLSRW weight
            w_inter = pinv2(
                ((Xk.T @ Xk) + (reg * (D.T @ D))), check_finite=False)
            x_weights = (w_inter @ Xk.T) @ Yk
            print("x_weights:", x_weights.shape)
            
            # Normalize weight
            x_weights /= np.sqrt(x_weights.T @ x_weights) + eps
            
            # Calculate the corresponding scores and loadings
            x_scores = Xk @ x_weights
            x_loadings = (Xk.T @ x_scores) / (x_scores.T @ x_scores)
            y_loadings = (Yk.T @ x_scores) / (x_scores.T @ x_scores)
            
            # Deflate X and Y
            Xk -= np.outer(x_scores, x_loadings)
            Yk -= np.outer(x_scores, y_loadings)
            print("Xk:", Xk.shape, "Yk:", Yk.shape)
            
            self.x_weights_[:, k] = x_weights[:, 0]
            self._x_scores[:, k] = x_scores[:, 0]
            self.x_loadings_[:, k] = x_loadings[:, 0]
            self.y_loadings_[:, k] = y_loadings[:, 0]

        # Compute transformation matrices
        self.x_rotations_ = np.dot(
            self.x_weights_,
            pinv2(np.dot(self.x_loadings_.T, self.x_weights_),
                  check_finite=False))
        
        self.coef_ = np.dot(self.x_rotations_, self.y_loadings_.T)
        self.coef_ = self.coef_ * self._y_std
        print("coef:", self.coef_.shape)
        return self
    
    def predict(self, X):
        return X @ self.coef_
    
    def score(self, X, y):
        y_pred = self.predict(X)
        return r2_score(y, y_pred)

In [ ]:
plsrw = PLSRW(n_components=2, reg=0)
plsrw.fit(X, y)
print(plsrw.score(X, y))

## PLS Group-and-Average

In [ ]:
def group_and_average(X, y, group_size=3):
    X_temp, y_temp = [], []
    
    # Multiply by 2 to enable the splits to be random
    num_sections = X.shape[0] / (group_size * 2)
    X_indices = np.arange(X.shape[0])
    subarray_indices = np.array(np.split(X_indices, num_sections, axis=0))
    
    # Shuffle each section so each time the split is run, the groups are different
    rng = np.random.default_rng()
    rng.shuffle(subarray_indices, axis=1)
    
    for subarray in subarray_indices:
        # Half each section to return to the original group size
        front_half, back_half = subarray[:group_size], subarray[group_size:]
        X_temp.append(np.mean(X[front_half], axis=0))
        X_temp.append(np.mean(X[back_half], axis=0))
        
        y_temp.append(np.mean(y[front_half]))
        y_temp.append(np.mean(y[back_half]))
    
    X_temp, y_temp = shuffle(X_temp, y_temp)
    return np.array(X_temp), np.array(y_temp)


def group_and_average_by_y(X, y, group_size=3):
    # Sort data by target
    sort_indices = np.argsort(y)
    X, y = X[sort_indices], y[sort_indices]
    
    return group_and_average(X, y, group_size)

In [ ]:
num_component = 2
train_scores, test_scores = [], []
pearsonr_scores, spearmanr_scores = [], []
coefs = []

for j in range(0, 10):
    X_group, y_group = group_and_average_by_y(X, y)
    mi = SelectKBest(mutual_info_regression, k=3000)
    X_group = mi.fit_transform(X_group, y_group)
    
    for i in range(0, 10):
        X_train, X_test, y_train, y_test = train_test_split(
            X_group, y_group, test_size=0.3, shuffle=True)

        pls = PLSRegression(n_components=num_component)
        pls.fit(X_train, y_train)

        y_train_pred = pls.predict(X_train)
        y_test_pred = pls.predict(X_test)[:, 0]

        train_scores.append(r2_score(y_train, y_train_pred))
        test_scores.append(r2_score(y_test, y_test_pred))
        pearsonr_scores.append(stats.pearsonr(y_test, y_test_pred)[0])
        spearmanr_scores.append(stats.spearmanr(y_test, y_test_pred)[0])
        coefs.append(pls.coef_)

avg_train_score, avg_test_score = np.mean(train_scores), np.mean(test_scores)
avg_pearsonr, avg_spearmanr = np.mean(pearsonr_scores), np.mean(spearmanr_scores)
avg_coef = np.mean(coefs, axis=0)

print(f'Measure: {selected_measure}')
print(f"Avg train score: {avg_train_score:.3f}")
print(f"Avg test score: {avg_test_score:.3f}")
print(f"Avg pearson: {avg_pearsonr:.3f}")
print(f"Avg spearman: {avg_spearmanr:.3f}")

## PLS Iteration with Percentage Split

In [ ]:
num_component = 2
train_scores, test_scores = [], []
pearsonr_scores, spearmanr_scores = [], []
coefs = []

for i in range(0, 1000):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, shuffle=True)
    
    pipe.fit(X_train, y_train)
    y_train_pred = pipe.predict(X_train)
    y_test_pred = pipe.predict(X_test)[:, 0]
    
    train_scores.append(r2_score(y_train, y_train_pred))
    test_scores.append(r2_score(y_test, y_test_pred))
    pearsonr_scores.append(stats.pearsonr(y_test, y_test_pred)[0])
    spearmanr_scores.append(stats.spearmanr(y_test, y_test_pred)[0])
    coefs.append(pls.coef_)
    
    if (i == 0):
        print(X_train.shape, X_test.shape)

avg_train_score, avg_test_score = np.mean(train_scores), np.mean(test_scores)
avg_pearsonr, avg_spearmanr = np.mean(pearsonr_scores), np.mean(spearmanr_scores)
avg_coef = np.mean(coefs, axis=0)

print(f"Avg train score: {avg_train_score:.4f}")
print(f"Avg test score: {avg_test_score:.4f}")
print(f"Avg pearson: {avg_pearsonr:.4f}")
print(f"Avg spearman: {avg_spearmanr:.4f}")

## Regression Scorer with r2

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score

def regression_scorer(reg, X, y):
    y_pred = reg.predict(X)[:, 0]
    
    return {'r2': r2_score(y, y_pred), 'pearsonr': stats.pearsonr(y, y_pred)[0],
            'explained_variance': explained_variance_score(y, y_pred)}

scoring = ['train_r2', 'test_r2', 'train_pearsonr', 'test_pearsonr', 'test_explained_variance']

## PLS with Expanded K-fold Cross-validation

In [ ]:
%%time
rkf = RepeatedKFold(n_splits=10, n_repeats=1, random_state=251183)
train_scores, test_scores = [], []
pearsonr_scores, spearmanr_scores = [], []
coefs = []

for train_index, test_index in rkf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    pipe = make_pipeline(StandardScaler(), PLSRegression(n_components=2))
    pipe.fit(X_train, y_train)
    y_train_pred = pipe.predict(X_train)
    y_test_pred = pipe.predict(X_test)[:, 0]
    
    train_scores.append(r2_score(y_train, y_train_pred))
    test_scores.append(r2_score(y_test, y_test_pred))
    pearsonr_scores.append(stats.pearsonr(y_test, y_test_pred)[0])
    spearmanr_scores.append(stats.spearmanr(y_test, y_test_pred)[0])
    coefs.append(pipe['plsregression'].coef_)

avg_train_score, avg_test_score = np.mean(train_scores), np.mean(test_scores)
avg_pearsonr, avg_spearmanr = np.mean(pearsonr_scores), np.mean(spearmanr_scores)
avg_coef = np.mean(coefs, axis=0)

print(f'Target: {selected_target}')
print(f"Avg train score: {avg_train_score:.2f}")
print(f"Avg test score: {avg_test_score:.2f}")
print(f"Avg pearson: {avg_pearsonr:.2f}")
print(f"Avg spearman: {avg_spearmanr:.2f}")

## PLS with Multivariate Y

In [ ]:
keys = Y.keys()
# keys = ['WISC_VSI', 'WISC_FRI', 'WISC_WMI', 'WISC_PSI', 'WISC_VCI']
# keys = 'WISC_BD_Scaled', 'WISC_Similarities_Scaled', 'WISC_MR_Scaled', 'WISC_DS_Scaled', 'WISC_Coding_Scaled', 'WISC_Vocab_Scaled', 'WISC_FW_Scaled', 'WISC_VP_Scaled', 'WISC_PS_Scaled', 'WISC_SS_Scaled'
y = np.array(list(map(Y.get, keys))).T

In [ ]:
%%time
pearsonr_scores = []

for train_index, test_index in rkf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    pipe.fit(X_train, y_train)
    y_test_pred = pipe.predict(X_test)
    
    pearsonr_temp = []
    for wisc_measure_num in range(0, y_test_pred.shape[1]):
        pearsonr_temp.append(stats.pearsonr(
            y_test[:, wisc_measure_num], y_test_pred[:, wisc_measure_num])[0])
        
    pearsonr_scores.append(pearsonr_temp)

pearsonr_scores = np.array(pearsonr_scores).T
avg_test_scores = np.mean(pearsonr_scores, axis=1)

for target, avg_test_score in zip(keys, avg_test_scores):
    print(f'Target: {target} | r: {avg_test_score:.2f}')

### Using permutation_test_score

In [ ]:
def regression_scorer(reg, X, y):
    y_pred = reg.predict(X)
    scores = []
    for measure in range(0, 6):
        scores.append(stats.pearsonr(y[:, measure], y_pred[:, measure])[0])
    
    return scores

In [ ]:
%%time
from common.binning import bin_by_age
from common.wisc import FSIQ, PRIMARY_INDICES

keys = FSIQ + PRIMARY_INDICES
y = np.array(list(map(Y.get, keys))).T

bins = bin_by_age(X, y, ages, y)
bin_1, bin_2, bin_3 = bins[0], bins[1], bins[2]
X_all = [X, bin_1[0], bin_2[0], bin_3[0]]
y_all = [y, bin_1[1], bin_2[1], bin_3[1]]
age_bin_label = ["All  ", "Bin 1", "Bin 2", "Bin 3"]

for X_cv, y_cv, bin_label in zip(X_all, y_all, age_bin_label):
    score, _, pvalue = permutation_test_score(
        pipe, X_cv, y_cv, cv=rkf, scoring=regression_scorer, n_permutations=3000, n_jobs=-1)
    print(f'Bin: {bin_label} | Target: {target} | Score: {score:.2f} | p-value: {pvalue:.4f}')
print('---')

## IQ Binning

In [ ]:
bins = bin_by_feature(X, y, y, 3)
bin_1, bin_2, bin_3 = bins[0], bins[1], bins[2]
print(f'Bin 1: {bin_1[0].shape} | Bin 2: {bin_2[0].shape} | Bin 3: {bin_3[0].shape}')

## Add Noise

In [ ]:
def generate_noise_samples(X, y, num_times):
    X_std = np.std(X, axis=0)
    
    for i in range(0, num_times):
        X_noisy = X + np.random.normal(0, X_std, X.shape)
        X, y = np.append(X, X_noisy, axis=0), np.append(y, y)
    
    return shuffle(X, y)

## Display PLS plots during or after training

In [ ]:
# Store the train, test, predicted train, and predicted test targets
y_train_store, y_train_pred_store = [], []
y_test_store, y_test_pred_store = [], []

train_scores.append(stats.pearsonr(y_train, y_train_pred)[0])
test_scores.append(stats.pearsonr(y_test, y_test_pred)[0])
y_train_store.append(y_train)
y_train_pred_store.append(y_train_pred)
y_test_store.append(y_test)
y_test_pred_store.append(y_test_pred)

# Select run most representative of final results
selected_k = -2
print(stats.pearsonr(y_train_store[selected_k], y_train_pred_store[selected_k])[0])
print(stats.pearsonr(y_test_store[selected_k], y_test_pred_store[selected_k])[0])

# Add LR to plot
y_train = y_train_store[selected_k].reshape(-1, 1)
y_test = y_test_store[selected_k].reshape(-1, 1)

y_train_pred_lr = LinearRegression().fit(y_train, y_train_pred).predict(y_train)
y_test_pred_lr = LinearRegression().fit(y_test, y_test_pred).predict(y_test)
print(y_train.shape, y_train_pred_lr.shape, y_test.shape, y_test_pred_lr.shape)

lin_reg_train_score = stats.pearsonr(y_train[:, 0], y_train_pred_lr)[0]
lin_reg_test_score = stats.pearsonr(y_test[:, 0], y_test_pred_lr)[0]
print("Train r:", lin_reg_train_score)
print("Test r:", lin_reg_test_score)

In [ ]:
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

lin_reg_train = LinearRegression().fit(y_train, y_train_pred)
y_train_pred_lin_reg = lin_reg_train.predict(y_train)

lin_reg_test = LinearRegression().fit(y_test, y_test_pred)
y_test_pred_lin_reg = lin_reg_test.predict(y_test)

lin_reg_train_score = lin_reg_train.score(y_train, y_train_pred)
lin_reg_test_score = lin_reg_test.score(y_test, y_test_pred)
print("Train r^2:", lin_reg_train_score)
print("Test r^2:", lin_reg_test_score)

In [ ]:
plt.scatter(y_train, y_train_pred, alpha=0.3, color='black')
plt.plot(y_train, y_train_pred_lin_reg, color='#897B61')
plt.title("Training Set")
plt.xlabel(f'True {selected_target}')
plt.ylabel(f'Predicted {selected_target}')
# plt.annotate(f"r-squared = {avg_train_score:.3f}", (6, 16))
plt.show()

In [ ]:
plt.scatter(y_test, y_test_pred, alpha=0.3, color='black')
plt.plot(y_test, y_test_pred_lin_reg, color='#897B61')
plt.title("Testing Set")
plt.xlabel(f'True {selected_target}')
plt.ylabel(f'Predicted {selected_target}')
# plt.annotate(f"r-squared = {lin_reg_test_score:.2f}", (60, 87))
plt.show()

In [ ]:
plt.figure(1, figsize=(10, 10))
plt.hist(x=test_scores, rwidth=0.95)
plt.title("Test Score Distribution")
plt.xlabel('Score')
plt.ylabel('Number of Scores')
plt.show()

## Display Yeo FC

In [ ]:
# To display each class of connections (within and between)
plt.figure(figsize=(10, 10))
plt.imshow(subject_fc)
a = np.zeros((11, 11))
a[fpn_indices] = subject_fc[fpn_indices]
plt.imshow(a)
b = np.zeros((8, 8))
b[np.triu_indices(8, k=1)] = subject_fc[dmn_indices]
plt.imshow(b)
plt.imshow(subject_fc[:11, 11:])

## Feature Selection (Poor Performance)

### Variance Threshold

In [ ]:
sel = VarianceThreshold(0.055)
X = sel.fit_transform(X)
print("X shape:", X.shape)

### Select k strongest connections

In [ ]:
def score_fc(X, y):
    # Take the strongest correlations regardless of sign
    sum_fc = np.absolute(np.sum(X, axis=0))
    return sum_fc

X = SelectKBest(score_fc, k=3400).fit_transform(X, y)
print("X shape:", X.shape)

### Display MI Before and After Transformation

In [ ]:
from sklearn.feature_selection import SelectKBest, mutual_info_regression

mi = SelectKBest(mutual_info_regression, k=1000)
X_mi = mi.fit_transform(X, y)
print("X_mi shape:", X_mi.shape)

mi_bin_1 = SelectKBest(mutual_info_regression, k=3000)
X_bin_1 = mi_bin_1.fit_transform(bin_1[0], bin_1[1])

mi_bin_2 = SelectKBest(mutual_info_regression, k=2000)
X_bin_2 = mi_bin_2.fit_transform(bin_2[0], bin_2[1])

mi_bin_3 = SelectKBest(mutual_info_regression, k=500)
X_bin_3 = mi_bin_3.fit_transform(bin_3[0], bin_3[1])

print(f'X_bin_1: {X_bin_1.shape} | X_bin_2: {X_bin_2.shape} | X_bin_3: {X_bin_3.shape}')

In [ ]:
from common.plotting import create_power_fc_matrix, plot_fc_matrix, plot_fc_graph

plot_fc_matrix(create_power_fc_matrix(X[1]), -1, 1)
plot_fc_matrix(create_power_fc_matrix(mi.inverse_transform(X_1[1].reshape(1, -1))), -1, 1)

## Download HBN Subjects

In [ ]:
from common.paths import BIOBANK_LABELS

downloads = '/imaging3/owenlab/wilson/Healthy'

temp = pd.read_csv(BIOBANK_LABELS + '/NoDiag_NoDL_Aug102021.csv').set_index('Identifiers')
temp = temp.drop(columns='File_Downloaded')
downloaded = [folder[4:] for folder in listdir(downloads) if folder.startswith("sub-")]
b = [1 for i in range(0, len(downloaded))]
downloaded = pd.DataFrame({'Identifiers': downloaded, 'File_Downloaded': b}).set_index('Identifiers')
# c = temp.index.tolist()
# for d in downloaded:
#     if d not in c:
#         print(d)
# display(downloaded)
# display(temp)

merged = pd.merge(temp, downloaded, left_index=True, right_index=True, how='left')
merged = merged.fillna(0, axis=1)
display(merged)
# merged.to_csv(BIOBANK_LABELS + '/NoDiag_NoDL_Aug102021.csv')

In [ ]:
import requests

def DownloadFile(url):
    local_filename = url.split('/')[-1]
    r = requests.get(url)
    print(r.status_code)
    if r.status_code == 404:
        return
    
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
    return 

## Check Comorbid ADHD

In [ ]:
diagnosis = subjects_with_labels[['assessment Diagnosis_ClinicianConsensus,DX_01',
                       'assessment Diagnosis_ClinicianConsensus,DX_02',
                       'assessment Diagnosis_ClinicianConsensus,DX_03',
                       'assessment Diagnosis_ClinicianConsensus,DX_04',
                       'assessment Diagnosis_ClinicianConsensus,DX_05',
                       'assessment Diagnosis_ClinicianConsensus,DX_06',
                       'assessment Diagnosis_ClinicianConsensus,DX_07',
                       'assessment Diagnosis_ClinicianConsensus,DX_08',
                       'assessment Diagnosis_ClinicianConsensus,DX_09',
                       'assessment Diagnosis_ClinicianConsensus,DX_10']]

diagnosis = diagnosis[diagnosis['assessment Diagnosis_ClinicianConsensus,DX_02'].isnull()]
diagnosis = diagnosis[diagnosis['assessment Diagnosis_ClinicianConsensus,DX_01'].str.contains('ADHD', regex=False)]

display(diagnosis)

## Compare Model Weights Across Age Bins

In [ ]:
from common.calculation import (
    calc_cosine_similarity, calc_norm_euclidean, compare_age_similarity)
from common.plotting import plot_age_comparisons

In [ ]:
all_ages_s, bin_1_s, bin_2_s, bin_3_s = np.clip(all_ages, np.min(all_ages), 0), np.clip(bin_1, np.min(bin_1), 0), np.clip(bin_2, np.min(bin_2), 0), np.clip(bin_3, np.min(bin_3), 0)

In [ ]:
comparisons = compare_age_similarity(all_ages_s, bin_1_s, bin_2_s, bin_3_s, calc_cosine_similarity)
np.fill_diagonal(comparisons, 1)
print(comparisons)
plot_age_comparisons(comparisons)

In [ ]:
distance_func = stats.spearmanr

print(distance_func(all_ages, bin_1))
print(distance_func(all_ages, bin_2))
print(distance_func(all_ages, bin_3))
print(distance_func(bin_1, bin_2))
print(distance_func(bin_1, bin_3))
print(distance_func(bin_2, bin_3))

## Read in subject IDs with FC using age

In [ ]:
from common.paths import POWER_FC

data_subject_ids = set()

for age in range(5, 19):
    age_dir = f'{POWER_FC}/Age{age}'
    
    if not exists(age_dir):
        continue
    
    curr_age_subjects = {folder[4:] for folder in listdir(age_dir) if folder.startswith("sub-")}
    data_subject_ids = data_subject_ids.union(curr_age_subjects)

data = pd.DataFrame(data_subject_ids, columns=['subject_id']).set_index('subject_id')

print(f'Number of subjects with FC: {len(data_subject_ids)}')

## Check WISC subtests against age

In [ ]:
# Grab columns
wisc_col = 'assessment WISC'
wisc_raw_measures = [
    'WISC_BD_Raw', 
    'WISC_Similarities_Raw', 
    'WISC_MR_Raw',
    'WISC_DS_Raw',
    'WISC_Coding_Raw',
    'WISC_Vocab_Raw',
    'WISC_FW_Raw',
    'WISC_VP_Raw',
    'WISC_PS_Raw',
    'WISC_SS_Raw',
]
wisc_scaled_measure = [
    'WISC_BD_Scaled', 
    'WISC_Similarities_Scaled', 
    'WISC_MR_Scaled',
    'WISC_DS_Scaled',
    'WISC_Coding_Scaled',
    'WISC_Vocab_Scaled',
    'WISC_FW_Scaled',
    'WISC_VP_Scaled',
    'WISC_PS_Scaled',
    'WISC_SS_Scaled',
]
wisc_measures = np.array([wisc_raw_measures, wisc_scaled_measure]).flatten('F')

clean_labels = clean_labels.dropna(subset=[f'assessment WISC,{measure}' for measure in wisc_measures])
wisc_measures_data = {measure: clean_labels[f'{wisc_col},{measure}'].astype(int).to_numpy() 
                      for measure in wisc_measures}
age = clean_labels['assessment Basic_Demos,Age'].astype(float)

print("Num WISC measures:", len(wisc_measures_data))
print("Num subjects:", len(age))

In [ ]:
for measure, data in wisc_measures_data.items():
    lin_reg = stats.linregress(age, data)
    print(f"{measure} r-squared:: {lin_reg.rvalue**2:.4f}")
    print(f"{measure} p-value: {lin_reg.pvalue}")
    print("--------------------------------------------")

## Univariate Ridge

In [ ]:
%%time
# X_cv = bin_1[0]
# y_cv = bin_1[1]
X_cv = X
y_cv = y
age_group = 'all'

estimators = [StandardScaler(), RidgeCV(alphas=[a for a in range(5000, 55000, 5000)], 
                                        scoring=regression_scorer, cv=rkf)]
pipe = make_pipeline(*estimators)
pipe.fit(X_cv, y_cv)
ridge_cv = pipe['ridgecv']

print(f'Target: {selected_target} | Alpha: {ridge_cv.alpha_} | Score: {ridge_cv.best_score_:.2f}')

estimators = [StandardScaler(), Ridge(alpha=ridge_cv.alpha_)]
pipe = make_pipeline(*estimators)

## Custom Permutation Test Score Function

### Version 1

In [ ]:
def bcustom_permutation_test_score(
    estimator, G_0, G_1, G_2, cv_0, cv_1, cv_2, n_permutations=100, scorer=None):
    
    G_0_scores, G_1_scores, G_2_scores, G_0_scores_perm, G_1_scores_perm, G_2_scores_perm = b_custom_permutation_test_score(
        clone(estimator), G_0[0], G_0[1], G_1[0], G_1[1], G_2[0], G_2[1], cv_0, cv_1, cv_2, n_permutations, scorer)
            
    G_0_score, G_1_score, G_2_score = np.mean(G_0_scores), np.mean(G_1_scores), np.mean(G_2_scores)
    
    G_0_scores_perm, G_1_scores_perm, G_2_scores_perm = np.array(G_0_scores_perm), np.array(G_1_scores_perm), np.array(G_2_scores_perm)
        
#     total_permutations = n_permutations * len(cv_0)
    total_permutations = n_permutations
    
    G_0_pvalue = calc_pvalue(G_0_scores_perm, G_0_score, total_permutations)
    G_1_pvalue = calc_pvalue(G_1_scores_perm, G_1_score, total_permutations)
    G_2_pvalue = calc_pvalue(G_2_scores_perm, G_2_score, total_permutations)

    return G_0_score, G_1_score, G_2_score, G_0_pvalue, G_1_pvalue, G_2_pvalue


def b_custom_permutation_test_score(estimator, X_0, y_0, X_1, y_1, X_2, y_2, cv_0, cv_1, cv_2, n_permutations, scorer):
    
    G_0_scores, G_1_scores, G_2_scores = [], [], []
    G_0_scores_perm, G_1_scores_perm, G_2_scores_perm = [], [], []
    
    for curr_cv in range(len(cv_0)):
        # Train and test on the in-group data
        cv_0_train, cv_0_test = cv_0[curr_cv][0], cv_0[curr_cv][1]
        X_0_train, X_0_test = X_0[cv_0_train], X_0[cv_0_test]
        y_0_train, y_0_test = y_0[cv_0_train], y_0[cv_0_test]
        
        estimator.fit(X_0_train, y_0_train)
        G_0_scores.append(scorer(estimator, X_0_test, y_0_test))
        
        # Test on the out-group data
        cv_1_test, cv_2_test = cv_1[curr_cv][1], cv_2[curr_cv][1]
        X_1_test, y_1_test = X_1[cv_1_test], y_1[cv_1_test]
        X_2_test, y_2_test = X_2[cv_2_test], y_2[cv_2_test]
        
        G_1_scores.append(scorer(estimator, X_1_test, y_1_test))
        G_2_scores.append(scorer(estimator, X_2_test, y_2_test))
        
        # Test on the shuffled out-group data
        rng = np.random.default_rng()
        G_0_scores_perm_temp, G_1_scores_perm_temp, G_2_scores_perm_temp = [], [], []
        for _ in range(n_permutations):
            y_0_test_perm = rng.permutation(y_0_test)
            y_1_test_perm = rng.permutation(y_1_test)
            y_2_test_perm = rng.permutation(y_2_test)
            
            G_0_scores_perm_temp.append(scorer(estimator, X_0_test, y_0_test_perm))
            G_1_scores_perm_temp.append(scorer(estimator, X_1_test, y_1_test_perm))
            G_2_scores_perm_temp.append(scorer(estimator, X_2_test, y_2_test_perm))
#             G_0_scores_perm.append(scorer(estimator, X_0_test, y_0_test_perm))
#             G_1_scores_perm.append(scorer(estimator, X_1_test, y_1_test_perm))
#             G_2_scores_perm.append(scorer(estimator, X_2_test, y_2_test_perm))
            
        G_0_scores_perm.append(np.mean(G_0_scores_perm_temp))
        G_1_scores_perm.append(np.mean(G_1_scores_perm_temp))
        G_2_scores_perm.append(np.mean(G_2_scores_perm_temp))
        
    return G_0_scores, G_1_scores, G_2_scores, G_0_scores_perm, G_1_scores_perm, G_2_scores_perm


In [ ]:
display_scores = []
# for bin_score in bin_test_scores:
#     display_scores.append([s['p_value'] for s in bin_score])
for _ in bin_test_labels:
    temp = []
    temp.append([s[1] for s in bin_score])
    temp = np.array(temp).flatten()
    display_scores.append(temp)

# print(len(bin_test_labels), len(display_scores))
results = pd.DataFrame(dict(zip(bin_test_labels, display_scores)))

ax = sns.boxplot(data=results, palette="Set2")
ax = sns.swarmplot(data=results, color="0.3", order=bin_test_labels)
ax.set_ylabel('Test Score (Pearson r)')
ax.set_xlabel('Age Bins')
ax.set_title('Cross Validation Results (Bin 1 -> Each Bin)')

### Version 2

In [ ]:
def custom_permutation_test_score(
    estimator, G_0, G_1, G_2, cv_0, cv_1, cv_2, n_permutations=100, scorer=None):
    
    G_scores, G_scores_perm = _custom_permutation_test_score(
        clone(estimator), G_0[0], G_0[1], G_1[0], G_1[1], G_2[0], G_2[1], cv_0, cv_1, cv_2, n_permutations, scorer)
    
    G_mean_scores, G_pvalues = [], []
    total_permutations = n_permutations
#     total_permutations = n_permutations * len(cv_0)

    for scores, scores_perm in zip(G_scores, G_scores_perm):
        mean_score = np.mean(scores)
        G_mean_scores.append(mean_score)
        
        scores_perm = np.array(scores_perm)
        pvalue = calc_pvalue(scores_perm, mean_score, total_permutations)
        G_pvalues.append(pvalue)

    return G_mean_scores, G_pvalues


def _custom_permutation_test_score(estimator, X_0, y_0, X_1, y_1, X_2, y_2, cv_0, cv_1, cv_2, n_permutations, scorer):
    # Index = group
    G_scores = [[], [], []]
    G_scores_perm = [[], [], []]
    
    for curr_cv in range(len(cv_0)):
        # Train and test on the in-group data
        cv_0_train, cv_0_test = cv_0[curr_cv][0], cv_0[curr_cv][1]
        X_0_train, X_0_test = X_0[cv_0_train], X_0[cv_0_test]
        y_0_train, y_0_test = y_0[cv_0_train], y_0[cv_0_test]
        
        estimator.fit(X_0_train, y_0_train)
        G_scores[0].append(scorer(estimator, X_0_test, y_0_test))
        
        # Test on the out-group data
        cv_1_test, cv_2_test = cv_1[curr_cv][1], cv_2[curr_cv][1]
        X_1_test, y_1_test = X_1[cv_1_test], y_1[cv_1_test]
        X_2_test, y_2_test = X_2[cv_2_test], y_2[cv_2_test]
        
        G_scores[1].append(scorer(estimator, X_1_test, y_1_test))
        G_scores[2].append(scorer(estimator, X_2_test, y_2_test))
        
        # Test on the shuffled out-group data
        rng = np.random.default_rng()
        G_scores_perm_temp = [[], [], []]
        X_tests = [X_0_test, X_1_test, X_2_test]
        y_tests = [y_0_test, y_1_test, y_2_test]
        
        for _ in range(n_permutations):
            for X_test, y_test, scores_temp in zip(X_tests, y_tests, G_scores_perm_temp):
#             for X_test, y_test, scores_temp in zip(X_tests, y_tests, G_scores_perm):
                y_test_perm = rng.permutation(y_test)
                scores_temp.append(scorer(estimator, X_test, y_test_perm))
        
        for score_perm, score_temp in zip(G_scores_perm, G_scores_perm_temp):
            score_perm.append(np.mean(score_temp))
        
    return G_scores, G_scores_perm

## ICC Out-Group

In [ ]:
%%time
import pingouin as pg

group_one = pd.DataFrame(np.squeeze(np.array(bin_2_coefs), axis=2))
group_one['group'] = 'g1'

group_two = pd.DataFrame(np.squeeze(np.array(bin_3_coefs), axis=2))
group_two['group'] = 'g2'

group_one_two = pd.concat([group_one, group_two])
icc_data = pd.melt(group_one_two, id_vars='group', var_name='connection', 
                   value_name='weight', ignore_index=False)
# display(icc_data)

icc = pg.intraclass_corr(data=icc_data, targets='connection', raters='group', ratings='weight').round(3)
icc.set_index("Type")

## Cross Prediction Print Results

In [ ]:
print(f'n={N_PERM},', selected_target)
print('r-values: {:.4f}, {:.4f}, {:.4f}'.format(*result[0]))
print('p-values: {:.4f}, {:.4f}, {:.4f}'.format(*result[2]))
print('mean permutation r-values: {:.4f}, {:.4f}, {:.4f}'.format(*[np.mean(perms) for perms in result[1]]))
print('---')

## Equal bin by feature

In [ ]:
def equal_bin_by_feature(X, y, feature, other_feature, num_bins=3):
    # Create the bins
    indices = np.arange(feature.shape[0])
    bin_indices = np.split(indices, num_bins)
    
    # Sort the data based on the feature
    sort_indices = np.argsort(feature)
    X, y = _select_data(X, y, sort_indices)
    
    # Apply the bins to the sorted data
    bins = [(X[bin_index], y[bin_index]) for bin_index in bin_indices]
    
    # Print bin statistics (age, sex, y)
    all_stats = [other_feature]
    for stat in all_stats:
        sorted_stat = stat[sort_indices]
        binned_stat = [sorted_stat[bin_index] for bin_index in bin_indices]
        
        for bin_num, feature_bin in enumerate(binned_stat):
            print(f'Bin {bin_num} Range: {np.min(feature_bin):.2f} -> {np.max(feature_bin):.2f}')
#             print(f'Bin {bin_num}: {np.unique(feature_bin, return_counts=True)}')
        print('---')
        
    return bins

## Display Distribution of Model Scores

In [ ]:
g = sns.displot(scores['test_r'], kde=True)
g.axes[0][0].axvline(np.mean(scores['test_r']), ls="--", label='mean', c='black')
g.axes[0][0].axvline(np.median(scores['test_r']), ls="--", label='median', c='red')
g.axes[0][0].set_title(f'r distribution ridge_{population}_{selected_target}_{age_group}')
g.axes[0][0].legend()

## Cross Validation with Bootstrapping

In [ ]:
from sklearn.utils import resample


def cross_validate_bootstrap(pipe, X, y, cv, scoring, n_samples):
    scores = []
    
    for train_index, test_index in cv.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        X_train, y_train = resample(X_train, y_train, n_samples=n_samples)
#         print(X_train.shape, y_train.shape)
        
        pipe.fit(X_train, y_train)
        scores.append(scoring(pipe, X_test, y_test))
    
    return scores

## Ridge with MultiOutputRegressor

In [ ]:
%%time
# X_cv = bin_1[0]
# y_cv = bin_1[1]
X_cv = X
y_cv = y
bin_label = 'all'

estimators = [StandardScaler(), MultiOutputRegressor(
    RidgeCV(alphas=[a for a in range(5000, 55000, 5000)], scoring=regression_scorer, cv=rkf))]
pipe = make_pipeline(*estimators)
pipe.fit(X_cv, y_cv)
ridge_cvs = pipe['multioutputregressor'].estimators_

for target, ridge_cv in zip(WISC_LEVEL[5], ridge_cvs):
    print(f'Target: {target} | Alpha: {ridge_cv.alpha_} | Score: {ridge_cv.best_score_:.2f}')

In [ ]:
for metric in SCORING:
    metric_values = scores[metric]
    print(f'Avg {metric}: {np.mean(metric_values):.3f}')

## Classifier Ridge for Predicting Sex

In [ ]:
%%time
from sklearn.linear_model import RidgeClassifierCV

X_cv = X
y_cv = sex
selected_target = "sex"
age_group = 'all'

estimators = [StandardScaler(), RidgeClassifierCV()]
pipe = make_pipeline(*estimators)

scores = cross_validate(pipe, X_cv, y_cv, cv=RKF_10_10, n_jobs=-1, 
                        return_train_score=False, return_estimator=False)

print(f'ridge_{population}_{selected_target}_{age_group}')
print(np.mean(scores['test_score']))

## Ridge with Bootstrapping

In [ ]:
%%time
from os.path import join
from common.binning import bin_data
from common.wisc import WISC_LEVEL
from common.paths import RIDGE_WEIGHTS, RIDGE_RESULTS
from common.scoring import bootstrap_permutation_test_score

results = []
targets = WISC_LEVEL[5]
target_alphas = [5000, 5000, 10000, 5000, 20000, 5000]

for target, target_alpha in zip(targets, target_alphas):
    y = Y[target]
    X_all, y_all, bin_labels = bin_data(X, y)
    
    for X_cv, y_cv, bin_label in zip(X_all, y_all, bin_labels):      
        estimators = [StandardScaler(), Ridge(alpha=target_alpha)]
        pipe = make_pipeline(*estimators)
        
        score, permutation_scores, pvalue = bootstrap_permutation_test_score(
            pipe, X_cv, y_cv, cv=RKF_10_10, scoring=unimetric_scorer, n_permutations=N_PERM, 
            n_jobs=-1, bootstrap_n=380)
        
        results.append({    
            'Model': 'ridge',
            'Population': population,
            'Target': target,
            'Bin': bin_label,
            'Alpha': target_alpha,
            'Score': score,
            'P-value': pvalue,
        })
        print(results[-1])

results_df = pd.DataFrame(results)
display(results_df.round(4))
# filename = f'ridge_cv_{population}.csv'
# results_df.to_csv(join(RIDGE_RESULTS, filename))

## Visualize Permutation Tests

In [ ]:
perms = np.array(perm_scores).reshape(9, N_PERM)
perms = pd.DataFrame(perms)

selected_results = results_df['Train'] == results_df['Test']

true_score = results_df[selected_results]
perm_score = perms[selected_results]
perm_score['Bin'] = [1, 2, 3]
perm_score = perm_score.melt(id_vars='Bin', var_name='Perm_num', value_name='r')

display(true_score)
display(perm_score)

In [ ]:
# Discrete variant
g = sns.displot(perm_score, x="r", hue="Bin", palette="Set2", element="step", col='Bin', legend=False)
g.fig.subplots_adjust(top=0.85)
g.fig.suptitle(f'Model: ridge, Target: {selected_target}, Train: Self, Test: Self, Population: ADHD, Num Perm: {N_PERM}')
for col_val, ax in g.axes_dict.items():
    bin_idx = col_val - 1
    ax.axvline(true_score['Score'].iloc[bin_idx], ls="--", color=sns.color_palette("Set2")[bin_idx])

In [ ]:
# Continuous variant
g = sns.displot(perm_score, x="r", hue="Bin", palette="Set2", kind="kde", fill=True, col='Bin')
for col_val, ax in g.axes_dict.items():
    bin_idx = col_val - 1
    ax.axvline(true_score['Score'].iloc[bin_idx], ls="--", color=sns.color_palette("Set2")[bin_idx])

In [ ]:
# Swarmplot variant
g = sns.catplot(x='Bin', y='r', data=perm_score, kind="swarm", palette="Set2", alpha=.75, s=4)
sns.swarmplot(x='Test', y='Score', data=true_score, ax=g.ax, color='black', alpha=1, marker='X', s=4)